# Coursera Capstone Week 3: Toronto Neighborhoods Project

There are 3 parts in this notebook. In part 1, I learn to do my first data scrape from the web using BeautifulSoup. In part 2, I clean the data obtained from the web scrape. In part 3, I cluster the neighborhoods of Toronto, Canada.

##### Notes:
In part 1, there are some lengthy print outs when viewing via github. You may want to use the side scroller to quickly pass through these after a quick look of each print out.

In part 3, folium does not render in github. It may be best to visit https://nbviewer.jupyter.org/ and copy and past the url to this github into the prompt for full view of outputs if you do not wish to download this notebook.

## Part 1: First Data Scrape of Wikipedia

In Part 1 I am learning how to scrape data from a website (hopefully anyway).

Website to be scraped: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

This is a list of neighborhoods by postal code within the city of Toronto, Canada. The aim is to scrape the wikipedia page; create a dataframe containing the data in tabular format, and then store the dataframe into a .csv file for further analysis.

Ultimately, these neighborhoods will be used to obtain gps coordinates to obtain further data for various venues through the Foursquare API and form cluster groups of these venues.

In [1]:
# import necessary libraries

# import numpy
import numpy as np

# import pandas
import pandas as pd

# import web scraping tools
from urllib.request import urlopen
from bs4 import BeautifulSoup

# library to handle JSON files
import json

# import geocoder
import geocoder

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# library to handle requests
import requests

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [2]:
# define the function that takes the desired url 
# and read/store its contents

def grab_html_contents(url):
    html = urlopen(url)
    html_page = html.read()
    html.close()
    soup = BeautifulSoup(html_page, 'html.parser')
    return soup

In [3]:
# Look at html page contents of desired page
# and parse through html tables to find the table desired

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html_content = grab_html_contents(url)

tables = html_content.find_all('table')
for table in tables:
    print(table.prettify())

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

In [4]:
# After inspecting the html, we see that 'wikitable sortable' 
# is the table we need so now we'll loop over the table
# data and perform the scrape

table = html_content.find('table', 
                     {'class': 'wikitable sortable'})

rows = table.find_all('tr')

# create .csv file for data to be saved to

file_name = 'toronto_postal_data.csv'
f = open(file_name,'w')

headers = 'PostalCode,Borough,Neighborhood\n'

f.write(headers)

# postal_data = []
# borough_data = []
# neighborhood_data = []

for index, row in enumerate(rows):
    cells = row.find_all('td')
    if len(cells) > 1:
        postal_data = cells[0].text.strip(' ')
        borough_data = cells[1].text.strip(' ')
        neighborhood_data = cells[2].text.strip()
        print('Obersvation {}'.format(index))
        print('Postal Code: ' + postal_data)
        print('Borough: ' + borough_data)
        print('Neighborhood: ' + neighborhood_data)
        f.write(postal_data + ',' + borough_data + ',' + neighborhood_data + '\n')

f.close()

Obersvation 1
Postal Code: M1A
Borough: Not assigned
Neighborhood: Not assigned
Obersvation 2
Postal Code: M2A
Borough: Not assigned
Neighborhood: Not assigned
Obersvation 3
Postal Code: M3A
Borough: North York
Neighborhood: Parkwoods
Obersvation 4
Postal Code: M4A
Borough: North York
Neighborhood: Victoria Village
Obersvation 5
Postal Code: M5A
Borough: Downtown Toronto
Neighborhood: Harbourfront
Obersvation 6
Postal Code: M6A
Borough: North York
Neighborhood: Lawrence Heights
Obersvation 7
Postal Code: M6A
Borough: North York
Neighborhood: Lawrence Manor
Obersvation 8
Postal Code: M7A
Borough: Downtown Toronto
Neighborhood: Queen's Park
Obersvation 9
Postal Code: M8A
Borough: Not assigned
Neighborhood: Not assigned
Obersvation 10
Postal Code: M9A
Borough: Etobicoke
Neighborhood: Islington Avenue
Obersvation 11
Postal Code: M1B
Borough: Scarborough
Neighborhood: Rouge
Obersvation 12
Postal Code: M1B
Borough: Scarborough
Neighborhood: Malvern
Obersvation 13
Postal Code: M2B
Borough: No

In [5]:
toronto_df = pd.read_csv('toronto_postal_data.csv')
print('Number of obersvations: {} \n'.format(toronto_df.shape[0]))
print('Number of features: {} \n'.format(toronto_df.shape[1]))
toronto_df.head()

Number of obersvations: 287 

Number of features: 3 



,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Part 2: Cleaning Data Scraped From Web

In this notebook I am cleaning the data obtained from wikipedia: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

First, import some necessary Python libraries.

Now read in the .csv file as a pandas dataframe and take a peak of the initial dataframe (raw scrape).

In [6]:
# create dataframe from .csv file that was scraped from wikipedia
toronto_df = pd.read_csv('toronto_postal_data.csv')

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Next I remove the 'Not assigned' boroughs. This eliminates all 'Not assigned' neighborhoods as well.  The info() method shows this.

In [7]:
# filter out 'Not assigned' values
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']

toronto_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210 entries, 2 to 285
Data columns (total 3 columns):
PostalCode      210 non-null object
Borough         210 non-null object
Neighborhood    210 non-null object
dtypes: object(3)
memory usage: 6.6+ KB


Just taking a peak to see the nice, neat structure of the data frame so far.

In [8]:
toronto_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Desired dataframe should have unique postal codes with the proper corresponding boroughs, and all neighborhoods within each postal code/borough pair should be merged into a list under the neighborhood column. To start this merge, I first create a unique list of tuples with unique postal code and its corresponding borough. Converting the zip object to a set removes all duplicates and then converting the set to a list allows me to iterate in the next portion of the merge.

In [9]:
# create list of tuples with unique postal code and corresponding unique borough
uni_list = zip(list(toronto_df['PostalCode']),list(toronto_df['Borough']))
uni_list = set(uni_list)
uni_list = list(uni_list)

To finish the merge I create a new, empty dataframe with proper column names. Then, I loop over the list of tuples defined above to merge the rows into the desired format as described above. Finally, I take a peak at the resulting dataframe.

In [10]:
# create new, filtered dataframe and take a (big) peak
toronto_cleaned_df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])

for i, tuple_ in enumerate(uni_list):
    postal_list = list(toronto_df[toronto_df['PostalCode'] == tuple_[0]]['Neighborhood'])
    toronto_cleaned_df.loc[i] = [tuple_[0], tuple_[1],
                                 ', '.join(postal_list)]


toronto_cleaned_df.head(50)

,PostalCode,Borough,Neighborhood
0,M4B,East York,"Woodbine Gardens, Parkview Hill"
1,M5A,Downtown Toronto,Harbourfront
2,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St..."
3,M6L,North York,"Downsview, North Park, Upwood Park"
4,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ..."
5,M6H,West Toronto,"Dovercourt Village, Dufferin"
6,M4X,Downtown Toronto,"Cabbagetown, St. James Town"
7,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel"
8,M6B,North York,Glencairn
9,M4P,Central Toronto,Davisville North


Checking the number of observations and features in the new, cleaned dataframe.

In [11]:
# check number of observations and features
toronto_cleaned_df.shape

(103, 3)

We see that there are 103 observations (103 unique postal codes).

### Assigning Lat./Long. Coords. to Postal Codes

Geocoder has not been working for me and, as stated in the assignment description, geocoder is sometimes not the most reliable. So I have decided to use the .csv file (https://cocl.us/Geospatial_data) provided that lists the postal codes with their accompanying latitude and longitude coordinates. So I create a dataframe from the .csv and concatenate that to the cleaned dataframe. I sort both dataframes by postal code (ascending), drop the postal codes from the lat/long dataframe and then concatenate the lat/long dataframe to the cleaned dataframe to ensure all coordinates are matching their corresponding postal code.

In [12]:
# read dataframe from geospatial .csv file (provided in assignement description) 
# and sort rows by postal code
lat_long_df = pd.read_csv('Geospatial_Coordinates.csv')
lat_long_df.sort_values(by=['Postal Code'], inplace=True)

lat_long_df.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [13]:
# drop 'Postal Code' column from lat_long_df, sort the cleaned dataframe, and concatenate
lat_long_df.drop(['Postal Code'], axis=1, inplace=True)
toronto_cleaned_df.sort_values(by=['PostalCode'], inplace=True) # sorts cleaned dataframe (index is out of order)
toronto_cleaned_df.reset_index(drop=True,inplace=True) # resets the index and drops the old index
toronto_geo_df = pd.concat([toronto_cleaned_df, lat_long_df], axis=1, sort=False) # concatenate dataframes' columns

toronto_geo_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Part 3: Clustering Neighborhoods in Toronto, ON, Canada

I will cluster neighborhoods based on postal codes; in the rendered map the tags should display the postal code followed by the borough the code is contained within.

In [14]:
# importing necessary libraries for clustering and visuals

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import folium for map rendering
import folium

In [15]:
# grabbing the lat/long coords for Toronto, Ontario, Canada

address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


Please note that since github does not display folium, if needed, you can visit https://nbviewer.jupyter.org/ and copy and past my github link into the url prompt. This will show the fully rendered maps without having to download the notebook and test it.

In [16]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, borough, postal in zip(toronto_geo_df['Latitude'], toronto_geo_df['Longitude'], toronto_geo_df['Borough'], toronto_geo_df['PostalCode']):
    label = '{}, {}'.format(postal, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Clustering Within Scarborough

I'm a fan of Simon and Garfunkle's "Scarborough Fair." Even though this song is not based on this location in Canada, it just reminds me of the song. So I'll investigate clusters in the Scarborough borough. Much of what follows is in keeping with the last lab of week 3.

In [17]:
# restrict the toronto dataset to a subset with Scarborough as the only borough
scarb_df = toronto_geo_df[toronto_geo_df['Borough'].str.contains('Scarborough')].reset_index(drop=True)
scarb_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(scarb_df['Latitude'], scarb_df['Longitude'], scarb_df['PostalCode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

You will need to specify your FourSquare credentials here to run:

In [ ]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Obtain neighborhood lat/long values.

In [20]:
neighborhood_latitude = scarb_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarb_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarb_df.loc[0, 'PostalCode'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of M1B are 43.806686299999996, -79.19435340000001.


Grab the top 100 venues in postal code M1B within 500 meters.

In [21]:
# type your answer here

LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=MSIPMKYMD3EQHDCXWOKUXA1FXYY03XPCXNINYK3DTVONOX4V&client_secret=00OL4CGUOU0FFIPUK1EDR55G3F0EMCRLGLBRVKVYAXJKTMX1&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

Sending get request.

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e5c6ab0c8cff2001b0a54b6'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

From the week 3 lab, we know that all the information is in the *items* key. Using the **get_category_type** function from week 3 lab.

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Cleaning the json and structuring it into a *pandas* dataframe.

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


See how many venues are returned.

In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


### Exploring Neighborhoods in Scarborough

Using function from week 3 lab to get nearby venues in Scarborough

In [26]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Next, store venues and list nearby neighborhoods.

In [27]:
# storing nearby venues and listing nearby neighborhoods
scarb_venues = getNearbyVenues(names=scarb_df['Neighborhood'],
                                   latitudes=scarb_df['Latitude'],
                                   longitudes=scarb_df['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


Checking size of venues dataframe.

In [28]:
print(scarb_venues.shape)

(88, 7)


Checking how many venues for neighborhoods associated with each postal code.

In [29]:
scarb_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Clarks Corners, Sullivan, Tam O'Shanter",15,15,15,15,15,15
"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",7,7,7,7,7,7
"East Birchmount Park, Ionview, Kennedy Park",5,5,5,5,5,5


Checking number of unique categories.

In [30]:
print('There are {} unique categories.'.format(len(scarb_venues['Venue Category'].unique())))

There are 54 unique categories.


### Analyze Neighborhoods

In [31]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarb_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarb_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,...,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


Check size of new dataframe.

In [32]:
scarb_onehot.shape

(88, 55)

See how often categories show up in each neighborhood group.

In [33]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,...,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000,0.000000,0.0,0.250000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.25,0.00,0.0,0.000000,0.000000,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.5,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.0,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.25,0.00,0.0,0.000000,0.000000,0.000000,0.000000
3,Cedarbrae,0.0,0.125,0.125,0.125000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.125000,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000,0.200,0.000000,0.0,0.000000,0.000000,0.2,0.1,...,0.0,0.000000,0.000000,0.00,0.00,0.1,0.000000,0.000000,0.000000,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.000,0.000,0.066667,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.066667,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.066667,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.5,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000,0.000,0.000000,0.0,0.000000,0.142857,0.0,0.0,...,0.0,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.142857
8,"East Birchmount Park, Ionview, Kennedy Park",0.0,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.2,...,0.0,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000
9,"Guildwood, Morningside, West Hill",0.0,0.000,0.000,0.000000,0.0,0.142857,0.000000,0.0,0.0,...,0.0,0.142857,0.000000,0.00,0.00,0.0,0.142857,0.000000,0.000000,0.000000


Checking size of new dataframe.

In [34]:
scarb_grouped.shape

(16, 55)

Printing each neighborhood group with top 5 common venues.

In [35]:
num_top_venues = 5

for hood in scarb_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarb_grouped[scarb_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4        American Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                       Park   0.5
1                 Playground   0.5
2        American Restaurant   0.0
3          Korean Restaurant   0.0
4  Latin American Restaurant   0.0


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                 venue  freq
0     Hakka Restaurant  0.12
1               Bakery  0.12
2                 Bank  0.12
3      Thai Restaurant  0.12
4  Fried Chicken Joint  0.12


----Clairlea, Golden Mile, Oakridge----
            venue  freq
0          

Using function from week 3 lab that sorts venues in descending order.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating dataframe that shows top 10 venues for each neighborhood group.

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Vietnamese Restaurant,Department Store
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Ice Cream Shop,Metro Station,Soccer Field,Intersection,Park,Bus Station,Fast Food Restaurant,Discount Store


Clustering neighborhood groups into 5 clusters.

In [38]:
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 1, 1, 1, 1, 4, 1, 1, 1])

Adding cluster column to Scarborough dataframe to identify cluster type.

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarb_merged = scarb_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarb_merged.dropna(inplace=True) # drops any NaN results that may pop up
scarb_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Vietnamese Restaurant,College Stadium,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1.0,Bar,Moving Target,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Spa,Intersection,Breakfast Spot,Rental Car Location,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium,General Entertainment
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Korean Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Vietnamese Restaurant,Department Store


As we notice the 'Cluster Labels' column contains a float type so we need to pass these entries as type int in for zip portion of for loop. This float type results when kmeans produces NaN values (which we dropped in previous cell).

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighborhood'], scarb_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

And there we have it! Neighborhood groups clustered according to postal code!